In [1]:
import os
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

import tensorflow as tf
# Il resto del tuo codice...

In [2]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

## ⚙️ Import Libraries

In [3]:
import os
from datetime import datetime

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
import random
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {tfk.__version__}")
print(f"GPU devices: {len(tf.config.list_physical_devices('GPU'))}")

TensorFlow version: 2.16.2
Keras version: 3.6.0
GPU devices: 1


## ⏳ Load the Data

In [4]:
#data = np.load("/content/drive/MyDrive/new_mars_clean_data.npz")
data = np.load("new_mars_clean_data.npz")
training_set = data["training_set"]
X_train = training_set[:, 0]
y_train = training_set[:, 1]
X_test = data["test_set"]

X_all = np.concatenate((X_train, X_test), axis=0)


print(f"Training X shape: {X_train.shape}")
print(f"Training y shape: {y_train.shape}")
print(f"Test X shape: {X_test.shape}")
print(f"All X shape: {X_all.shape}")

Training X shape: (2505, 64, 128)
Training y shape: (2505, 64, 128)
Test X shape: (10022, 64, 128)
All X shape: (12527, 64, 128)


## 🛠️ Train and Save the Model

In [5]:
# Add color channel and rescale pixels between 0 and 1
X_train = X_train[..., np.newaxis] / 255.0
X_test = X_test[..., np.newaxis] / 255.0

input_shape = X_train.shape[1:]
num_classes = len(np.unique(y_train))

print(f"Input shape: {input_shape}")
print(f"Number of classes: {num_classes}")

Input shape: (64, 128, 1)
Number of classes: 5


In [6]:
# Definition of hyperparameters

data_params = {
    'batch_size': 32,
    'input_shape': (64, 128, 3),
    'num_classes': 5,
    'seed': 42
}

HYPERPARAMETERS = {
    "BATCH_SIZE": 32,
    "EPOCHS": 10,
    "LEARNING_RATE": 0.00001,
    "LEARNING_DESCENT_PATIENCE": 20,
    "LEARNING_DESCENT_FACTOR": 0.5,
    "EARLY_STOPPING_PATIENCE": 30,
    "DROPOUT": 0.4,
    "LAYERS_FINE_TUNE": 0,
    "MODEL_NAME": "boh",
}

Try to build an autoencoder to do good initialization

In [8]:
import tensorflow as tf
from tensorflow.keras import layers as tfkl

# Residual Block
def residual_block(x, filters, kernel_size=3, activation='relu', name=''):
    x = tfkl.Conv2D(filters, kernel_size, padding='same', kernel_initializer='he_normal', name=name + '_conv1')(x)
    x = tfkl.BatchNormalization(name=name + '_bn1')(x)
    x = tfkl.Activation(activation, name=name + '_activation1')(x)

    x = tfkl.Conv2D(filters, kernel_size, padding='same', kernel_initializer='he_normal', name=name + '_conv2')(x)
    x = tfkl.BatchNormalization(name=name + '_bn2')(x)
    x = tfkl.Activation(activation, name=name + '_activation2')(x)
    return x

# U-Net with Attention and Residual Blocks
def get_segmentation_model(input_shape=input_shape, num_classes=num_classes, dropout_rate=0.0):
    inputs = tfkl.Input(shape=input_shape, name='input_layer')

    # Downsampling Path
    d1 = residual_block(inputs, 64, name='down_block1')
    p1 = tfkl.MaxPooling2D(pool_size=(2, 2), name='pool1')(d1)

    d2 = residual_block(p1, 128, name='down_block2')
    p2 = tfkl.MaxPooling2D(pool_size=(2, 2), name='pool2')(d2)

    d3 = residual_block(p2, 256, name='down_block3')
    p3 = tfkl.MaxPooling2D(pool_size=(2, 2), name='pool3')(d3)

    d4 = residual_block(p3, 512, name='down_block4')
    p4 = tfkl.MaxPooling2D(pool_size=(2, 2), name='pool4')(d4)

    # Bottleneck with Dropout and Dilated Convolutions
    b = residual_block(p4, 1024, name='bottleneck')  # Aumenta i filtri per il bottleneck
    b = tfkl.Dropout(dropout_rate, name='bottleneck_dropout')(b)

    # Upsampling Path with Attention
    u4 = tfkl.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same', name='up_block4_upsample')(b)
    u4 = residual_block(u4, 512, name='up_block4')

    u3 = tfkl.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same', name='up_block3_upsample')(u4)
    u3 = residual_block(u3, 256, name='up_block3')

    u2 = tfkl.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same', name='up_block2_upsample')(u3)
    u2 = residual_block(u2, 128, name='up_block2')

    u1 = tfkl.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same', name='up_block1_upsample')(u2)
    u1 = residual_block(u1, 64, name='up_block1')
    
    decoded = tfkl.Conv2D(1, 1, padding="same", activation="sigmoid")(u1)

    # Model
    model = tf.keras.Model(inputs=inputs, outputs=decoded, name='Advanced_UNet')
    return model



In [17]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os

#callbacks
class VizCallback(tf.keras.callbacks.Callback):
    def __init__(self, image_paths, label_paths, frequency=40):
        super().__init__()
        self.image_paths = image_paths
        self.label_paths = label_paths
        self.frequency = frequency

    def on_epoch_end(self, epoch, logs=None):
        if epoch % self.frequency == 0:  # Visualize only every "frequency" epochs
            image = self.image_paths[261]
            label = self.label_paths[261]
            image1 = tf.expand_dims(image, 0)
            pred = self.model.predict(image1, verbose=0)
            y_pred = tf.math.argmax(pred, axis=-1)
            y_pred = y_pred.numpy()

            colors = ['red', 'green', 'blue', 'orange', 'purple']  # Adjust colors as needed
            class_labels = ['background', 'soil', 'bedrock', 'sand', 'big_rock']  # Add your class labels
            cmap = ListedColormap(colors)

            # Number of images to display
            num_images = 3

            # Select random indices

            # Create the plot
            fig, axes = plt.subplots(num_images, 4, figsize=(12, 20))  # Adjust width to fit legend


                # Display the grayscale image
            axes[0, 0].imshow(image, cmap='gray')
            axes[0, 0].set_title(f"Image 261")
            axes[0, 0].axis('off')

                # Display the mask with the custom colormap
            axes[0, 1].imshow(label, cmap=cmap, vmin=0, vmax=len(colors) - 1)
            axes[0, 1].set_title(f"True 261")
            axes[0, 1].axis('off')

            axes[0, 2].imshow(y_pred[0], cmap=cmap, vmin=0, vmax=len(colors) - 1)
            axes[0, 2].set_title(f"prediction 261")
            axes[0, 2].axis('off')

            patches = [mpatches.Patch(color=colors[j], label=class_labels[j]) for j in range(len(colors))]
            axes[0, 3].legend(
                    handles=patches,
                    loc='center',
                    title="Classes",
                    bbox_to_anchor=(0.5, 0.5),
                )
            axes[0, 3].axis('off')

            image = self.image_paths[28]
            label = self.label_paths[28]
            image1 = tf.expand_dims(image, 0)
            pred = self.model.predict(image1, verbose=0)
            y_pred = tf.math.argmax(pred, axis=-1)
            y_pred = y_pred.numpy()

                # Display the grayscale image
            axes[1, 0].imshow(image, cmap='gray')
            axes[1, 0].set_title(f"Image 28")
            axes[1, 0].axis('off')

                # Display the mask with the custom colormap
            axes[1, 1].imshow(label, cmap=cmap, vmin=0, vmax=len(colors) - 1)
            axes[1, 1].set_title(f"True 28")
            axes[1, 1].axis('off')

            axes[1, 2].imshow(y_pred[0], cmap=cmap, vmin=0, vmax=len(colors) - 1)
            axes[1, 2].set_title(f"prediction 28")
            axes[1, 2].axis('off')

            patches = [mpatches.Patch(color=colors[j], label=class_labels[j]) for j in range(len(colors))]
            axes[1, 3].legend(
                    handles=patches,
                    loc='center',
                    title="Classes",
                    bbox_to_anchor=(0.5, 0.5),
                )
            axes[1, 3].axis('off')

            r=random.randint(0, 200)
            image = self.image_paths[r]
            label = self.label_paths[r]
            image1 = tf.expand_dims(image, 0)
            pred = self.model.predict(image1, verbose=0)
            y_pred = tf.math.argmax(pred, axis=-1)
            y_pred = y_pred.numpy()

            axes[2, 0].imshow(image, cmap='gray')
            axes[2, 0].set_title(f"Image {r}")
            axes[2, 0].axis('off')

                # Display the mask with the custom colormap
            axes[2, 1].imshow(label, cmap=cmap, vmin=0, vmax=len(colors) - 1)
            axes[2, 1].set_title(f"True {r}")
            axes[2, 1].axis('off')

            axes[2, 2].imshow(y_pred[0], cmap=cmap, vmin=0, vmax=len(colors) - 1)
            axes[2, 2].set_title(f"prediction {r}")
            axes[2, 2].axis('off')

            patches = [mpatches.Patch(color=colors[j], label=class_labels[j]) for j in range(len(colors))]
            axes[2, 3].legend(
                    handles=patches,
                    loc='center',
                    title="Classes",
                    bbox_to_anchor=(0.5, 0.5),
                )
            axes[2, 3].axis('off')


            plt.tight_layout()
            plt.show()

#image_callback = ImageCallback(data=X_all, log_dir='callback_images', frequency=1)
#viz_callback = VizCallback(X_all, X_all)

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
# Definizione dell'early stopping
early_stopping = EarlyStopping(
    monitor='loss',       # Monitora la validation loss
    patience=30,              # Numero di epoche senza miglioramento prima di fermarsi
    restore_best_weights=True # Ripristina i pesi del modello alla migliore epoca
)

# Esempio di utilizzo
reduce_on_plateau = ReduceLROnPlateau(
        monitor='reconstruction_accuracy',
        factor=0.5,
        patience=7,
        min_lr=0.00001,
        verbose=1
    )

In [18]:
print(f"Shape X_all: {X_all.shape}")
if(X_all.shape == (12527, 64, 128)):
    X_all = X_all[..., np.newaxis].astype('float32')/ 255.0

input_shape = X_all.shape[1:]  # Ottieni la shape delle immagini (64, 128, 1)
print(f"Shape input_shape: {X_all.shape}")


#creo autoencoder
autoencoder = get_segmentation_model(input_shape=input_shape)
autoencoder.summary()


Shape X_all: (12527, 64, 128, 1)
Shape input_shape: (12527, 64, 128, 1)


Model: "Advanced_UNet"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 64, 128, 1)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ down_block1_conv1 (Conv2D)      │ (None, 64, 128, 64)    │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ down_block1_bn1                 │ (None, 64, 128, 64)    │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ down_block1_activation1         │ (None, 64, 128, 64)    │             0 │
│ (Activation)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ down_block1_conv2 (Conv2D)      │ (None, 64, 128, 64)    │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ down_block1_bn2                 │ (None, 64, 128, 64)    │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ down_block1_activation2         │ (None, 64, 128, 64)    │             0 │
│ (Activation)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool1 (MaxPooling2D)            │ (None, 32, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ down_block2_conv1 (Conv2D)      │ (None, 32, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ down_block2_bn1                 │ (None, 32, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ down_block2_activation1         │ (None, 32, 64, 128)    │             0 │
│ (Activation)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ down_block2_conv2 (Conv2D)      │ (None, 32, 64, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ down_block2_bn2                 │ (None, 32, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ down_block2_activation2         │ (None, 32, 64, 128)    │             0 │
│ (Activation)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool2 (MaxPooling2D)            │ (None, 16, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ down_block3_conv1 (Conv2D)      │ (None, 16, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ down_block3_bn1                 │ (None, 16, 32, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ down_block3_activation1         │ (None, 16, 32, 256)    │             0 │
│ (Activation)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ down_block3_conv2 (Conv2D)      │ (None, 16, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ down_block3_bn2                 │ (None, 16, 32, 256)    │         1,02

 Total params: 27,920,705 (106.51 MB)

 Trainable params: 27,908,929 (106.46 MB)

 Non-trainable params: 11,776 (46.00 KB)

In [19]:
# Definisci una metrica di accuratezza per la ricostruzione
def reconstruction_accuracy(y_true, y_pred):
    # Puoi definire l'accuratezza come la percentuale di pixel con un errore inferiore a una soglia (es. 0.1)
    threshold = 0.02
    correct_predictions = tf.reduce_mean(
        tf.cast(tf.abs(y_true - y_pred) < threshold, tf.float32)
    )
    return correct_predictions

def ssim_loss(y_true, y_pred):
    return 1 - tf.image.ssim(y_true, y_pred, max_val=1.0)

from tensorflow.keras.optimizers import Adam

# Compilare il modello
autoencoder.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=ssim_loss,  # L1 Loss
    metrics=[reconstruction_accuracy]  # Aggiungi la metrica personalizzata
)


In [21]:
# Training del modello con la callback
history = autoencoder.fit(
    X_all, X_all,
    epochs=200,
    batch_size=32,
    shuffle=True,
    callbacks=[early_stopping, reduce_on_plateau]  # Aggiunta della callback
)


# Salva il modello se necessario
autoencoder.save("autoencoder_per_giuseppe073.h5")

Epoch 1/200
392/392 ━━━━━━━━━━━━━━━━━━━━ 173s 441ms/step - loss: 0.0478 - reconstruction_accuracy: 0.8554 - learning_rate: 1.2500e-04
Epoch 2/200
392/392 ━━━━━━━━━━━━━━━━━━━━ 174s 443ms/step - loss: 0.0478 - reconstruction_accuracy: 0.8554 - learning_rate: 1.2500e-04
Epoch 3/200
392/392 ━━━━━━━━━━━━━━━━━━━━ 174s 443ms/step - loss: 0.0477 - reconstruction_accuracy: 0.8554 - learning_rate: 1.2500e-04
Epoch 4/200
392/392 ━━━━━━━━━━━━━━━━━━━━ 174s 444ms/step - loss: 0.0476 - reconstruction_accuracy: 0.8555 - learning_rate: 1.2500e-04
Epoch 5/200
392/392 ━━━━━━━━━━━━━━━━━━━━ 174s 445ms/step - loss: 0.0476 - reconstruction_accuracy: 0.8557 - learning_rate: 1.2500e-04
Epoch 6/200
392/392 ━━━━━━━━━━━━━━━━━━━━ 174s 445ms/step - loss: 0.0475 - reconstruction_accuracy: 0.8560 - learning_rate: 1.2500e-04
Epoch 7/200
392/392 ━━━━━━━━━━━━━━━━━━━━ 174s 445ms/step - loss: 0.0475 - reconstruction_accuracy: 0.8562 - learning_rate: 1.2500e-04
Epoch 8/200
392/392 ━━━━━━━━━━━━━━━━━━━━ 175s 446ms/step - los